In [1]:
from zetamarkets_py.types import Asset, Side, OrderOptions, OrderType
from zetamarkets_py.client import Client
import anchorpy
import os
from datetime import datetime, timedelta
from solana.rpc.commitment import Confirmed, Processed

endpoint = os.getenv("ENDPOINT", "https://api.mainnet-beta.solana.com")
print(endpoint.split(".com")[0])

wallet = anchorpy.Wallet.local()  # get local filesystem keypair wallet

https://api.mainnet-beta.solana


In [2]:
asset = Asset.SOL
client = await Client.load(endpoint=endpoint, commitment=Processed, wallet=wallet, assets=[asset])

In [3]:
import websockets
import json

In [6]:
import asyncio
import time
from typing import List
from zetamarkets_py.orderbook import Orderbook

from zetamarkets_py.types import Order


class MarketMaker:
    def __init__(self, client: Client, asset: Asset, current_open_orders: List[Order]) -> None:
        self.client = client
        self.asset = asset
        self._is_quoting = False
        self.fair_price = None
        self.offset_bps = 5
        self.edge_bps = 5
        self.quote_size = 0.001

        # get the best bid in the open orders
        self.bid_price = max([o.info.price for o in current_open_orders if o.side == Side.Bid], default=None)
        self.ask_price = min([o.info.price for o in current_open_orders if o.side == Side.Ask], default=None)

        self.EDGE_REQUOTE_THRESHOLD = 0.25
        self.TIF_DURATION = 120

    @classmethod
    async def load(cls, endpoint, wallet, asset, commitment=Confirmed):
        client = await Client.load(endpoint=endpoint, commitment=commitment, wallet=wallet, assets=[asset])
        oo = await client.fetch_open_orders(asset)
        return cls(client, asset, oo)

    async def subscribe_fair_price(self):
        async with websockets.connect("wss://fstream.binance.com/ws/solusdt@bookTicker") as ws:
            await ws.send(
                json.dumps({"method": "SUBSCRIBE", "params": ["solusdt@bookTicker"], "id": int(time.time() * 1000)})
            )
            first_resp = await ws.recv()
            async for msg in ws:
                try:
                    ticker = json.loads(msg)
                    if ticker["e"] == "bookTicker":
                        vwap = (float(ticker["b"]) * int(ticker["B"]) + float(ticker["a"]) * int(ticker["A"])) / (
                            int(ticker["B"]) + int(ticker["A"])
                        )
                        self.fair_price = vwap
                        await self.update_quotes()
                except ValueError as e:
                    print(e)
                except Exception as e:
                    print(e)

    async def subscribe_zeta_price(self):
        async def on_bid_update(orderbook: Orderbook):
            print(f"Best bid: {orderbook._get_l2(1)}")

        async def on_ask_update(orderbook: Orderbook):
            print(f"Best ask: {orderbook._get_l2(1)}")

        await asyncio.gather(
            self.client.subscribe_orderbook(self.asset, Side.Bid, on_bid_update),
            self.client.subscribe_orderbook(self.asset, Side.Ask, on_ask_update),
        )

    async def update_quotes(self):
        if self._is_quoting:
            return
        if self.fair_price is None:
            print("Missing fair price")
            return
        # Skip requote if the fair price is within the edge
        fair_price = self.fair_price * (1 + self.offset_bps / 10000)
        if self.bid_price is not None and self.ask_price is not None:
            # Get the latest mark price and calculate bid/ask prices
            current_mid_price = (self.bid_price + self.ask_price) / 2
            edge = current_mid_price * (self.edge_bps / 10000)

            # If the fair price is within the edge, don't update the quotes
            # print(f"Edge ratio: {abs(fair_price - current_mid_price) / edge}")
            if abs(fair_price - current_mid_price) < edge * self.EDGE_REQUOTE_THRESHOLD:
                # print(f"Skipping quote since fair price is within edge: {abs(fair_price - current_mid_price)} vs {edge}")
                return

        print(f"NBBO price: {self.fair_price}")
        print(f"Internal price: {fair_price}")

        # Requote
        bid_price = fair_price * (1 - self.edge_bps / 10000)
        ask_price = fair_price * (1 + self.edge_bps / 10000)

        # Set order options
        expiry_ts = int((datetime.now() + timedelta(seconds=self.TIF_DURATION)).timestamp())
        order_opts = OrderOptions(
            expiry_ts=expiry_ts, order_type=OrderType.PostOnly, client_order_id=69
        )  # TODO: postOnly

        # Execute quote!
        self._is_quoting = True
        try:
            print(f"Requoting: {bid_price} @ {ask_price}")
            await client.replace_quote(asset, bid_price, self.quote_size, ask_price, self.quote_size, order_opts)
            self.bid_price = bid_price
            self.ask_price = ask_price
        except Exception as e:
            raise e
        finally:
            self._is_quoting = False

    async def run(self):
        try:
            await asyncio.gather(await self.subscribe_fair_price(), await self.subscribe_zeta_price())
        finally:
            await client.cancel_orders_for_market(self.asset)

In [7]:
endpoint = os.getenv("ENDPOINT", "https://api.mainnet-beta.solana.com")
wallet = anchorpy.Wallet.local()  # get local filesystem keypair wallet
mm = await MarketMaker.load(endpoint, wallet, Asset.SOL, Processed)

await mm.run()

INFO:zetamarkets_py.client.Client:Replacing SOL orders: 0.001x Bid @ $19.062630807463634, 0.001x Ask @ $19.081702974354542
INFO:solanaweb3.rpc.httprpc.HTTPClient:Transaction sent to https://api.mainnet-beta.solana.com. Signature hE8SMHTqLPFb1TpnhL9gUcYsVH12y5fhkURcX96SYDPeg4GLbQMBfFrBB2X1ZpjxCjWMxE2iLzGECEixjzpDwut: 


NBBO price: 19.062635573122527
Internal price: 19.072166890909088
Requoting: 19.062630807463634 @ 19.081702974354542


INFO:zetamarkets_py.client.Client:Replacing SOL orders: 0.001x Bid @ $19.059708989718402, 0.001x Ask @ $19.07877823332992


NBBO price: 19.05971375464684
Internal price: 19.06924361152416
Requoting: 19.059708989718402 @ 19.07877823332992


INFO:solanaweb3.rpc.httprpc.HTTPClient:Transaction sent to https://api.mainnet-beta.solana.com. Signature 2MYCgYUWc7cPVUk1ESaBu6sB8LVBnM9ydMahEBTfinuX9hA1ftGeT9NzjSN2CEcbSDif9576otf42uRTt53HFL7F: 
INFO:zetamarkets_py.client.Client:Replacing SOL orders: 0.001x Bid @ $19.05610376310465, 0.001x Ask @ $19.075169399686047
INFO:solanaweb3.rpc.httprpc.HTTPClient:Transaction sent to https://api.mainnet-beta.solana.com. Signature 62tdypSewzZtkCAuwGLkmmRt62RkXe1bajhw7kR6imMv2r9wDuadwzL3dC62d7NkMygkujRWBj9MkXLkMH8kBQKQ: 


...


NBBO price: 19.077218117854002
Internal price: 19.086756726912927
Requoting: 19.07721334854947 @ 19.096300105276384


INFO:zetamarkets_py.client.Client:Replacing SOL orders: 0.001x Bid @ $19.077223802121434, 0.001x Ask @ $19.096310569307146



NBBO price: 19.077228571428577
Internal price: 19.08676718571429
Requoting: 19.077223802121434 @ 19.096310569307146



INFO:zetamarkets_py.client.Client:Cancelling all orders for SOL
INFO:solanaweb3.rpc.httprpc.HTTPClient:Transaction sent to https://api.mainnet-beta.solana.com. Signature 5PG3No3gT41LaqiRsHGgz1SCCY4ejqwtytz6GUapV2xRfS6M74me7EGBfwk5fpsFMav73VewQnSVGn8wxgtsNVGb: 


ConnectionClosedError: sent 1011 (unexpected error) keepalive ping timeout; no close frame received